In [2]:
import sys
import cv2
import os 
import numpy as np
import random

In [3]:
!pip install import-ipynb

In [4]:
sys.path.append('/Users/sandeep/opt/anaconda3/lib/python3.7/site-packages')
import import_ipynb
import object_detection
import stereo_to_3d

importing Jupyter notebook from object_detection.ipynb
importing Jupyter notebook from stereo_to_3d.ipynb


# Global variable setup

In [5]:
master_path_to_dataset = "/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10" # ** need to edit this **
directory_to_cycle_left = "left-images"     # edit this if needed
directory_to_cycle_right = "right-images"   # edit this if needed

# set this to a file timestamp to start from (empty is first example - outside lab)
# e.g. set to 1506943191.487683 for the end of the Bailey, just as the vehicle turns

skip_forward_file_pattern = ""; # set to timestamp to skip forward to

crop_disparity = False; # display full or cropped disparity image
pause_playback = False; # pause until key press after each image


# resolve full directory location of data set for left / right images

full_path_directory_left =  os.path.join(master_path_to_dataset, directory_to_cycle_left);
full_path_directory_right =  os.path.join(master_path_to_dataset, directory_to_cycle_right);

left_file_list = sorted(os.listdir(full_path_directory_left));

# setup the disparity stereo processor to find a maximum of 128 disparity values
# (adjust parameters if needed - this will effect speed to processing)

max_disparity = 128;
stereoProcessor = cv2.StereoSGBM_create(0, max_disparity, 21);

# Used for segmenting object from background
mask_threshold = 0.3

# Looping over image files

- Load the image file
- For left and right images 
    - Pass the image to Mask RCNN and do object detection
        - Should return the follwingg 
            - 
            - [1:{class_name:'dog'  , left-top: (x,y) ,right-bottom: (x1,y1), mask:}]
    - Remove the background and only keep the objects in the image 
            - Get back and white mask,(DONE) 
            - get object only(DONE)
- Find the disparity map of the using the segmented Left and Right image file
- Overlay the mask with countour with disaprity map to find 
    - Center of countour region 
    - Mean of countour region
    - Median of countour region
- Edit project disparity to 3d such that
       - for each detected object it only calculates 3d point of given coords (center)
       - x,y,z
             - You do not need x and y 
             - You only need compute and return z 
- Pass the image to Mask RCNN module agian to draw the rectangle and z value on it
        - draw_pred needs: image,class_name,(left-top-coord),(right-bottom-coord), colour
        - Thus, you need to store 

In [6]:
def get_filtering_mask(class_mask):
    return (class_mask > mask_threshold)

In [7]:
def get_rgb_mask(img,filtering_mask):
    rgb_mask = np.zeros(img.shape, np.uint8)
    rgb_mask[top:bottom+1, left:right+1][filtering_mask] = np.array(([255.0,255.0,255.0])).astype(np.uint8)
    return rgb_mask

In [8]:
def get_countour(class_mask):
    filtering_mask = get_filtering_mask(class_mask)
    filtering_mask = filtering_mask.astype(np.uint8)
    contours, hierarchy = cv2.findContours(filtering_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [9]:
def get_center_disparity(left,top,class_mask,dispairty_map):
    countours = get_countour(class_mask)
    cnt = countours[0]
    M = cv2.moments(cnt)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    x,y = cX + left , cY + top  
    obj_center_pixel_disparity = dispairty_map[y,x]
    return obj_center_pixel_disparity

In [10]:
def get_mean_disparity(left,top,right,bottom,class_mask, imgL,disparity_map):
    filtering_mask = get_filtering_mask(class_mask)
#     rgb_mask = get_rgb_mask(imgL,filtering_mask)
    
#     mask = cv2.cvtColor(rgb_mask, cv2.COLOR_BGR2GRAY)
    mask= np.zeros(disparity_map.shape, np.uint8)
    mask[top:bottom+1, left:right+1][filtering_mask] = np.array(([255.0])).astype(np.uint8)
    obj_mean_disparity = cv2.mean(disparity_map, mask=mask)
    return obj_mean_disparity[0]

In [12]:
object_detection.conf_threshold = 0.87

In [15]:
counter = 0
for filename_left in left_file_list:

    # from the left image filename get the correspondoning right image
    filename_right = filename_left.replace("_L", "_R");
    full_path_filename_left = os.path.join(full_path_directory_left, filename_left);
    full_path_filename_right = os.path.join(full_path_directory_right, filename_right);


    # for sanity print out these filenames
    print(full_path_filename_left);
    print(full_path_filename_right);
    print();

    if ('.png' in filename_left) and (os.path.isfile(full_path_filename_right)) :

        # read left and right images and display in windows
        # N.B. despite one being grayscale both are in fact stored as 3-channel
        # RGB images so load both as such
        imgL = cv2.imread(full_path_filename_left, cv2.IMREAD_COLOR)
        imgR = cv2.imread(full_path_filename_right, cv2.IMREAD_COLOR)
        print("-- files loaded successfully");
        print();
        
        segmented_imgL, detected_objects_list_L = object_detection.process_img(imgL)
        segmented_imgR, detected_objects_list_R = object_detection.process_img(imgR)
        # remember to convert to grayscale (as the disparity matching works on grayscale)
        # N.B. need to do for both as both are 3-channel images
        grayL = cv2.cvtColor(segmented_imgL,cv2.COLOR_BGR2GRAY);
        grayR = cv2.cvtColor(segmented_imgR,cv2.COLOR_BGR2GRAY);
        
        # compute disparity image from undistorted and rectified stereo images
        # that we have loaded
        disparity = stereoProcessor.compute(grayL,grayR);
        dispNoiseFilter = 5; # increase for more agressive filtering
        cv2.filterSpeckles(disparity, 0, 4000, max_disparity - dispNoiseFilter);
        _, disparity = cv2.threshold(disparity,0, max_disparity * 16, cv2.THRESH_TOZERO);
        disparity_scaled = (disparity / 16.).astype(np.uint8);
        
        
        for detection_number in range(len(detected_objects_list_L)):
            detected_object = detected_objects_list_L[detection_number]
            #Get object detection results
            class_name  = detected_object['class_name']
            class_mask = detected_object['class_mask']
            left , top = detected_object['left'],detected_object['top']
            right, bottom = detected_object['right'], detected_object['bottom']
            
#             #get disparity value at the center of the object
            center_disparity = get_center_disparity(left ,
                                                    top,
                                                    class_mask, 
                                                    disparity_scaled)
            
            #get the average disparity value relative to the object segment region
#             mean_disparity = get_mean_disparity(left,
#                                                 top,
#                                                 right,
#                                                 bottom,
#                                                 class_mask,
#                                                 imgL,
#                                                 disparity_scaled)
            
            
            #Given a disparity value calculate the z value
            distance = stereo_to_3d.project_disparity_to_3d(center_disparity)
            
            if distance != 0:
                imgL = object_detection.drawPred(imgL,
                                                 class_name,
                                                 left,top,
                                                 right,bottom,
                                                 (255, 178, 50),
                                                 distance
                                                )
                                            
        cv2.imwrite('/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/results_mask_rcnn_0.87/'+filename_left, imgL)          
#         cv2.imshow('Distance detection',imgL)
        
#         key = cv2.waitKey(40 * (not(pause_playback))) & 0xFF; # wait 40ms (i.e. 1000ms / 25 fps = 40 ms)
#         if (key == ord('x')):       # exit
#             break; # exit
#         elif (key == ord('s')):     # save
#             cv2.imwrite("sgbm-disparty.png", disparity_scaled);
#             cv2.imwrite("left.png", imgL);
#             cv2.imwrite("right.png", imgR);
#         elif (key == ord('c')):     # crop
#             crop_disparity = not(crop_disparity);
#         elif (key == ord(' ')):     # pause (on next frame)
#             pause_playback = not(pause_playback);
#     else:
#         print("-- files skipped (perhaps one is missing or not PNG)");
#         print();


/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942473.484027_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942473.484027_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942474.483193_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942474.483193_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942475.481834_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942475.481834_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942476.480930_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942503.475460_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942503.475460_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942504.475323_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942504.475323_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942505.475215_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942505.475215_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942506.475136_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942533.475109_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942533.475109_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942534.475102_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942534.475102_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942535.475090_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942535.475090_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942536.475079_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942563.483813_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942563.483813_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942564.483124_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942564.483124_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942565.482748_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942565.482748_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942566.482532_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942593.475299_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942593.475299_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942594.475307_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942594.475307_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942595.475435_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506942595.475435_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506942596.475391_L.png
/Users/sandeep/Desktop/Homework2019/Compute

...

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943818.379335_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943818.379335_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943819.379346_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943819.379346_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943820.379354_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943820.379354_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943821.379476_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943848.381248_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943848.381248_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943849.380639_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943849.380639_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943850.380336_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943850.380336_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943851.380287_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943878.392877_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943878.392877_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943879.391243_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943879.391243_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943880.390120_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943880.390120_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943881.389334_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943908.379324_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943908.379324_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943909.379474_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943909.379474_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943910.379617_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943910.379617_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943911.379767_L.png
/Users/sandeep/Desktop/Homework2019/Compute

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943938.380059_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943938.380059_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943939.380148_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943939.380148_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943940.380325_L.png
/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/right-images/1506943940.380325_R.png

-- files loaded successfully

/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/TTBB-durham-02-10-17-sub10/left-images/1506943941.380408_L.png
/Users/sandeep/Desktop/Homework2019/Compute

In [14]:
%debug

> <string>(5)project_disparity_to_3d()

ipdb> up
> <ipython-input-13-17eec1251685>(66)<module>()
     64 
     65             #Given a disparity value calculate the z value
---> 66             distance = stereo_to_3d.project_disparity_to_3d(mean_disparity)
     67 
     68             if distance != 0:

ipdb> up
*** Oldest frame
ipdb> mean_disparity
(18.322597566093158, 0.0, 0.0, 0.0)
ipdb> exit


# Histrogram equalization Experiment

- Select few dark/bright images with
    - Shadows (DONE)
    - Over-exposed images (DONE)
    - Take a look at histogram 
           - Before equalization (NOT NEEDED)
           - After equalization(NOT NEEDED)
- Object Detection
    -  Write a function that performs detection on images and saves it (DONE)
          - Without equalization (DONE)
              - just pass the image straight through
          - With equalization (DONE)
                - Write a fuction that performs historgram equalization for RGB space (DONE)
                - With Gaussian Blur
          - With adaptive equalization(DONE)
                - With Gaussian Blur
          - Compare the difference (DONE) 
- Disparity map (DONE)
    - Write a function that takes a gray scale img produces disparity map
        - Without equalization
            - dont convert the image into gray scale 
            - produce disaprity and save it
            - With gaussian blur
        - With equalization 
            - convert the image into gray scale 
            - produce disaprity and save it
            - With gaussian blur 
- Distance
    - Write a function that produces images with distance coming from 
        - equalized disparity (DONE)
        - non-equalized disparity (DONE

In [5]:
def hisEqulColor(img):
    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    cv2.equalizeHist(channels[0],channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
    return img

In [39]:
def adaptiveHisEqulColor(img):
    bgr = img 
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    
    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(16,16))

    lab_planes[0] = clahe.apply(lab_planes[0])

    lab = cv2.merge(lab_planes)

    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return bgr

In [40]:
bad_img_l_dir = '/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/bad_images/left-images'
bad_img_r_dir = full_path_directory_right
bad_img_l_file_list = sorted(os.listdir(bad_img_l_dir))
for filename_left in bad_img_l_file_list:
    if filename_left != '.DS_Store':
        # from the left image filename get the correspondoning right image
        filename_right = filename_left.replace("_L", "_R");
        full_path_filename_left = os.path.join(bad_img_l_dir, filename_left);
        full_path_filename_right = os.path.join(full_path_directory_right, filename_right);

        imgL = cv2.imread(full_path_filename_left, cv2.IMREAD_COLOR)
        imgL_copy = imgL[:]
        imgR = cv2.imread(full_path_filename_right, cv2.IMREAD_COLOR)
        imgR_copy = imgR[:]

        # remember to convert to grayscale (as the disparity matching works on grayscale)
        # N.B. need to do for both as both are 3-channel images

        grayL = cv2.cvtColor(imgL_copy,cv2.COLOR_BGR2GRAY);
        grayR = cv2.cvtColor(imgR_copy,cv2.COLOR_BGR2GRAY);
        
        #uncomment below to use histogram equalized  gray scale images to find disparity
#         grayL = cv2.equalizeHist(grayL)
#         grayR = cv2.equalizeHist(grayR)
        
        #uncomment below to use adaptive histogram equalized gray scale images to find disparity
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16,16))
        grayL = clahe.apply(grayL)
        grayR = clahe.apply(grayR)
    
        # uncomment below to use gaussian blur on image that make disparity map
#         grayL = cv2.GaussianBlur(grayL,(5,5),1)
#         grayR = cv2.GaussianBlur(grayR,(5,5),1)
        
        #uncomment below to use bilateral-filtering on images that make disparity
        grayL = cv2.bilateralFilter(grayL,5,75,75)
        grayR = cv2.bilateralFilter(grayR,5,25,25)
        
        # uncomment below to use median filter on images that make dispartiy map 
#         grayL = cv2.medianBlur(grayL,9)
#         grayR = cv2.medianBlur(grayR,9)
        
        #The code below generates disparity map
        disparity = stereoProcessor.compute(grayL,grayR);
        dispNoiseFilter = 5; # increase for more agressive filtering
        cv2.filterSpeckles(disparity, 0, 4000, max_disparity - dispNoiseFilter);
        _, disparity = cv2.threshold(disparity,0, max_disparity * 16, cv2.THRESH_TOZERO);
        disparity_scaled = (disparity / 16.).astype(np.uint8);
        
##############################################################################################################################################
        #uncomment for Object detection with histogram equalization
#         imgL = hisEqulColor(imgL)
        
        #uncomment for object detection with adaptive histogram equalization
        imgL = adaptiveHisEqulColor(imgL)
        
        # uncomment below to use gaussian blur 
#          imgL = cv2.GaussianBlur(imgL, (5,5) ,1)

        #uncomment below to use bilateral-filtering
        imgL = cv2.bilateralFilter(imgL,5,75,75)
        
        #uncomment below to use median filter
#         imgL = cv2.medianBlur(imgL,9)
        
        detected_object_list = object_detection.process_image(imgL)
        for detected_object in detected_object_list:
                center_of_bounding_box_coord = detected_object['center']
                distance = stereo_to_3d.project_disparity_to_3d(disparity_scaled,
                                                        max_disparity,
                                                        center_of_bounding_box_coord)
                imgL = object_detection.drawPred(imgL,
                                             detected_object['class_name'],
                                             detected_object['left_top'][0],detected_object['left_top'][1],
                                             detected_object['right_bottom'][0],detected_object['right_bottom'][1],
                                             (255, 178, 50),
                                             distance)

        file_name = 'HE_B_'+filename_left
        a_he_dir = '/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/bad_images/object_det_exp/A_HE/16_B_75'
        disparity_images_dir = '/Users/sandeep/Desktop/Homework2019/ComputerVision Homework/bad_images/object_det_exp/A_HE/16_B_75/disparity/'
        path = os.path.join(a_he_dir,file_name)
        path_disparity = os.path.join(disparity_images_dir,file_name)
        cv2.imwrite(path,imgL)
        cv2.imwrite(path_disparity,disparity_scaled)

TypeError: 'NoneType' object is not subscriptable